In [1]:
from pkg.data_collecting.load_data import *

JOINT_NUM = 13
CONVERTED_PATH = CONVERTED_PATH_DEFAULT

gtimer = GlobalTimer()
# ## Load Global params
DATASET_LIST = sorted(os.listdir(CONVERTED_PATH))
trainset_num = None
if trainset_num is not None:
    TRAINSET_LIST =  DATASET_LIST[:trainset_num]
    TESTSET_LIST = DATASET_LIST[trainset_num:]
else:
    TRAINSET_LIST = ['20201214-165211', '20201216-021416', '20201218-024611']
    TESTSET_LIST = ['20201208-121454', '20201212-232318', '20201213-061207']
data_loader = DataLoader(JOINT_NUM)
train_set = data_loader.get_dataset_args(TRAINSET_LIST, JOINT_NUM)
N_train = len(train_set)
test_set = data_loader.get_dataset_args(TESTSET_LIST, JOINT_NUM)
N_test = len(test_set)
print(f'Train/Test: {N_train}, {N_test}')

Train/Test: 59522, 19215


In [2]:
from pkg.training.model import *

# Create an instance of the model
model = ResNetModelTP()

loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        # training=True is only needed if there are layers with different
        # behavior during training versus inference (e.g. Dropout).
        predictions = model(images, training=True)
        loss = loss_object(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)
    

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
@tf.function
def test_step(images, labels):
    # training=False is only needed if there are layers with different
    # behavior during training versus inference (e.g. Dropout).
    predictions = model(images, training=False)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)

In [3]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logpath = os.path.join('logs','gradient_tape',current_time)
train_log_dir = os.path.join(logpath, 'train')
test_log_dir = os.path.join(logpath, 'test')
model_log_dir = os.path.join(logpath, 'model_{}/')
train_summary_writer = tf.summary.create_file_writer(train_log_dir)
test_summary_writer = tf.summary.create_file_writer(test_log_dir)
shutil.copy(os.path.join('.','pkg','training','model.py' ), logpath)
print(f'Log path: {logpath}')

Log path: logs/gradient_tape/20210110-211851


In [4]:
EPOCHS_S = 0
EPOCHS_E = 30
BATCH_SIZE = 32
LOG_STEP = 100

for epoch in range(EPOCHS_S, EPOCHS_E):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    
    random.shuffle(train_set)
    i_step = 0
    scene_batch, success_batch = [], []
    for scene_tuple in train_set:
        i_step += 1
        scene_data, success, skey = load_scene_data(*scene_tuple)
        scene_batch.append(scene_data)
        success_batch.append(success)
        if i_step%BATCH_SIZE==0:
            cbox, cbox_m, ccyl, ccyl_m, ibox, ibox_m, gbox, gbox_m = data_loader.separate_dat(np.array(scene_batch, dtype=np.float32))
            train_step([cbox, cbox_m, ccyl, ccyl_m, ibox, ibox_m, gbox, gbox_m], np.array(success_batch, dtype=np.int))
            scene_batch, success_batch = [], []
        if i_step%LOG_STEP==0:
            print("train step - {}/{}        ".format(i_step, N_train), end = '\r')
    with train_summary_writer.as_default():
        tf.summary.scalar('loss', train_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)

    i_step = 0
    scene_batch, success_batch = [], []
    for scene_tuple in test_set:
        i_step += 1
        scene_data, success, skey = load_scene_data(*scene_tuple)
        scene_batch.append(scene_data)
        success_batch.append(success)
        if i_step%BATCH_SIZE==0:
            cbox, cbox_m, ccyl, ccyl_m, ibox, ibox_m, gbox, gbox_m = data_loader.separate_dat(np.array(scene_batch, dtype=np.float32))
            test_step([cbox, cbox_m, ccyl, ccyl_m, ibox, ibox_m, gbox, gbox_m], np.array(success_batch,dtype=np.int))
            scene_batch, success_batch = [], []
        if i_step%LOG_STEP==0:
            print("test step - {}/{}        ".format(i_step, N_test), end = '\r')
    with test_summary_writer.as_default():
        tf.summary.scalar('loss', test_loss.result(), step=epoch)
        tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
            
    model.save(model_log_dir.format(epoch + 1))

    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_accuracy.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_accuracy.result() * 100}'
    )

NotFoundError:  No algorithm worked!
	 [[node res_net_model_tp/res_net/conv_block3d_4/res1a_branch2b/Conv3D (defined at /home/tamp/Projects/tamp_etasl/eTaSL/pkg/training/model.py:114) ]] [Op:__inference_train_step_28666]

Errors may have originated from an input operation.
Input Source operations connected to node res_net_model_tp/res_net/conv_block3d_4/res1a_branch2b/Conv3D:
 res_net_model_tp/res_net/conv_block3d_4/activation_24/IdentityN (defined at /home/tamp/Projects/tamp_etasl/eTaSL/pkg/training/model.py:112)

Function call stack:
train_step


### To Do
* input stage batch norm 없애보기 - 미미한 향상
* dense depth 조정, add layer를 concat으로 - trainset 위주의 미미한 향상
* input stage kernel size 1로, feature size down, dropout - 상당한 향성
* dropout 증강 - 미미한 변화
* stage0~5 last feature size 절반 - 무변화
* box preconv 통합, feature size 추가 축소 - 성능 하락
* box preconv 분리 - 성능 복구
* feature size 절반, batch size 128 - 미미한 성능 하락
* feature size 복구, batch size 64 - 성능 복구 ~ 77% (20201224-122148)
* input non-batchnorm 확대 - 차이 없음 ~ 77% (20201224-165237)
* 인풋 수정, input bn 복구 - 성능 하락 ~ 73% (20210106-150839) - 데이터 로딩 실수, 분리된 파일들에 기존의 내부 카운팅으로 중복 로드
* 인풋 joint limit 상대값, 데이터 디버그 - 하락 상태, 큰 차이 없음 -74% (20210107-033852) 
* 인풋 피쳐만 절반 - 전혀 학습X (20210107-172454)
* 전반적 피쳐 확대 - 전전과 비등 (20210107-230440)
* elu/sigmoid 적용 - 수렴 속도만 빨라짐 (20210108-064925)
* swish 적용 -  (20210108-202908)


* input stage batch norm 더 없애보기
* fix data 비주얼로 검증해보기
* non-fix train 결과로 fixed 결과에 테스트 해보기

### train log
* dense bn for end stage: saturated output
* dense batch norm for each stage: 74% for train, saturated for test
* dense linear for each stage: 73% for train, saturated for test
* dense bn relu - dense linear for each stage: 74% for train, saturated for test
* dense bn relu - dense linear for each stage (size down): 74% for train, saturated for test
* separate input - dense bn relu - dense linear for each stage : 50epoch - 98.5% for train, max 74% for test
* separate input - dense bn relu - dense linear for each stage : 60k data - 95% for train, 75% for test
* 32 batch - OOM
* dense^2 relu/sigmoid for each stage: 
* dense^2 relu/sigmoid x2 for each stage: 

In [ ]:
import numpy as np

In [4]:
XX= np.zeros((5,5,5,10))

In [13]:
XX[:,:,:,:] = list(range(10))

In [14]:
XX[0,2,1]

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

* use_bias=True
```
INFO:tensorflow:Assets written to: logs/gradient_tape/20201221-143930/model.h5/assets
Epoch 1, Loss: 0.6301059126853943, Accuracy: 67.04729461669922, Test Loss: 0.6941890120506287, Test Accuracy: 60.01933288574219
INFO:tensorflow:Assets written to: logs/gradient_tape/20201221-143930/model.h5/assets
Epoch 2, Loss: 0.5641522407531738, Accuracy: 70.55376434326172, Test Loss: 1.1243842840194702, Test Accuracy: 58.13466262817383
INFO:tensorflow:Assets written to: logs/gradient_tape/20201221-143930/model.h5/assets
Epoch 3, Loss: 0.4620269536972046, Accuracy: 77.59195709228516, Test Loss: 0.6866365671157837, Test Accuracy: 59.51997756958008
INFO:tensorflow:Assets written to: logs/gradient_tape/20201221-143930/model.h5/assets
Epoch 4, Loss: 0.43595388531684875, Accuracy: 79.04708862304688, Test Loss: 2.0774402618408203, Test Accuracy: 52.75450897216797
INFO:tensorflow:Assets written to: logs/gradient_tape/20201221-143930/model.h5/assets
Epoch 5, Loss: 0.4291463792324066, Accuracy: 80.04749298095703, Test Loss: 0.5646138787269592, Test Accuracy: 67.20361328125
INFO:tensorflow:Assets written to: logs/gradient_tape/20201221-143930/model.h5/assets
Epoch 6, Loss: 0.4223867654800415, Accuracy: 79.8807601928711, Test Loss: 0.84770268201828, Test Accuracy: 54.961341857910156
INFO:tensorflow:Assets written to: logs/gradient_tape/20201221-143930/model.h5/assets
Epoch 7, Loss: 0.41381269693374634, Accuracy: 80.5881118774414, Test Loss: 1.0013036727905273, Test Accuracy: 56.2983283996582
train step - 6500/19801  
```

* use_bias=False - 큰 차이 없음
```
INFO:tensorflow:Assets written to: logs/gradient_tape/20201221-161530/model.h5/assets
Epoch 1, Loss: 0.6417393088340759, Accuracy: 66.24393463134766, Test Loss: 0.6525431871414185, Test Accuracy: 64.90013122558594
INFO:tensorflow:Assets written to: logs/gradient_tape/20201221-161530/model.h5/assets
Epoch 2, Loss: 0.5980429649353027, Accuracy: 69.05315399169922, Test Loss: 0.6825580596923828, Test Accuracy: 55.91172409057617
INFO:tensorflow:Assets written to: logs/gradient_tape/20201221-161530/model.h5/assets
Epoch 3, Loss: 0.5077642202377319, Accuracy: 75.03031158447266, Test Loss: 0.7368553280830383, Test Accuracy: 62.12950897216797
INFO:tensorflow:Assets written to: logs/gradient_tape/20201221-161530/model.h5/assets
Epoch 4, Loss: 0.47730717062950134, Accuracy: 76.97049713134766, Test Loss: 1.066005825996399, Test Accuracy: 56.459407806396484
INFO:tensorflow:Assets written to: logs/gradient_tape/20201221-161530/model.h5/assets
Epoch 5, Loss: 0.4632640480995178, Accuracy: 78.0820541381836, Test Loss: 0.769783079624176, Test Accuracy: 62.580543518066406
train step - 12800/19801     
```

* use_biase=False, robot position sampled